In [1]:
import pandas as pd
import re

In [2]:
INPUT_FILE = "./resources/Histology Clearing Requests.xlsx"
input_sheet = pd.read_excel(INPUT_FILE, sheet_name="Histology Clearing Requests")
print(input_sheet)

                           LabTrack ID#                               Name  \
0                        753117, 753118            web-form@smartsheet.com   
1    729431 730677 732965 732966 737812            web-form@smartsheet.com   
2                                720956            web-form@smartsheet.com   
3                                711256            web-form@smartsheet.com   
4                                720935            web-form@smartsheet.com   
..                                  ...                                ...   
383                              691388  mathew.summers@alleninstitute.org   
384                              672373   kanghoon.jung@alleninstitute.org   
385                              663046   kanghoon.jung@alleninstitute.org   
386                              667574   kanghoon.jung@alleninstitute.org   
387                              673206   kanghoon.jung@alleninstitute.org   

     Requester Name              Group  \
0    Mathew Summers  

c:\Users\mae.moninghoff\AppData\Local\miniconda3\envs\aind-data-migration\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
mapping = {}

for index, row in input_sheet.iterrows():
    requester = row["Requester Name"]
    ids_string = str(row["LabTrack ID#"])
    ids_list = re.findall(r'[0-9]+', ids_string)
    ids_list = [id for id in ids_list if len(id) == 6]
    print(requester, ids_list)
    if requester not in mapping.keys():
        mapping[requester] = ids_list
    elif requester in mapping.keys():
        mapping[requester].extend(ids_list)

print(mapping)

Mathew Summers ['753117', '753118']
Anna Lakunina ['729431', '730677', '732965', '732966', '737812']
Xinxin Yin ['720956']
Xinxin Yin ['711256']
Xinxin Yin ['720935']
Xinxin Yin ['716870']
Mathew Summers ['741534', '741535', '741536']
Mathew Summers ['743709', '743801']
Mathew Summers ['745297']
Polina Kosillo ['736019']
Polina Kosillo ['742880']
Polina Kosillo ['742321']
Smrithi Sunil ['747772', '737963']
Mathew Summers ['743800']
Naveen Ouellette ['737828']
Xinxin Yin ['722680']
Xinxin Yin ['699983']
Galen Lynch ['713510']
Mathew Summers ['701752', '701753', '701755']
nan []
Xinxin Yin ['726477']
Xinxin Yin ['713379']
Xinxin Yin ['713377']
Naveen Ouellette ['747810', '747813']
Severine Durand ['725108']
Severine Durand ['729286']
Severine Durand ['729287']
Polina Kosillo ['737038']
Polina Kosillo ['737042']
Mathew Summers ['737043']
Xinxin Yin ['699984']
Xinxin Yin ['716866']
Anna Lakunina ['716878', '716876']
Anna Lakunina ['728538', '731004']
Anna Lakunina ['725805', '733124']
Anna

In [4]:
from aind_data_access_api.document_db_ssh import DocumentDbSSHClient, DocumentDbSSHCredentials


output_df = pd.DataFrame(columns=["Requester", "LabTrack ID#", "Asset Name"])
found_records = 0

for investigator, ids in mapping.items():
    print(investigator, ids)
    for id in ids:
        print(f"checking for {id}")
        credentials = DocumentDbSSHCredentials()

        with DocumentDbSSHClient(credentials=credentials) as doc_db_client:
            query = {
                'name': {
                    '$regex': 'smartspim', 
                    '$options': 'i'
                }, 
                'data_description.subject_id': id
            }
            projection = {
                "name": 1, "created": 1, "location": 1, "data_description": 1, "last_modified": 1
            }
            document_count = doc_db_client.collection.count_documents(query)
            response = list(doc_db_client.collection.find(filter=query, projection=projection))
            print(f"found {document_count} records")
            if document_count:
                found_records += 1

        for record in response:
            asset_name = record["name"]
            new_df = pd.DataFrame([{"Requester": investigator, "LabTrack ID#": id, "Asset Name": asset_name}])
            output_df = pd.concat([output_df, new_df], ignore_index=True)

print(output_df)
print(f"found {found_records}/{sum([len(ids) for investigators, ids in mapping.items()])} records")
output_df.to_excel("output.xlsx", index=False)


c:\Users\mae.moninghoff\AppData\Local\miniconda3\envs\aind-data-migration\Lib\site-packages\paramiko\pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\mae.moninghoff\AppData\Local\miniconda3\envs\aind-data-migration\Lib\site-packages\paramiko\transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


Mathew Summers ['753117', '753118', '741534', '741535', '741536', '743709', '743801', '745297', '743800', '701752', '701753', '701755', '737043', '734248', '737040', '728853', '742628', '728852', '732918', '741208', '732915', '734247', '730931', '732181', '732182', '724927', '725326', '721316', '727325', '727324', '725145', '726090', '716952', '725024', '726089', '713974', '715876', '716453', '716454', '713975', '713971', '724016', '724017', '725022', '710627', '710628', '722627', '722628', '710625', '710626', '721832', '687503', '713603', '714633', '714634', '712556', '713602', '709392', '710240', '710241', '711323', '711324', '712557', '568105', '651050', '708396', '708398', '709391', '699316', '699317', '699318', '701750', '701751', '701754', '701121', '702320', '702321', '699834', '699835', '701120', '698034', '698035', '692912', '692911', '692913', '692908', '693979', '693980', '693977', '693978', '692907', '696668', '696669', '692909', '698927', '698928', '693196', '695721', '694